#### Importing Python packages 

In [36]:
# Importing Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [37]:
# Getting current working directory
print(os.getcwd())

# Fetching folder and subfolder of the event data
filepath = os.getcwd() + '/event_data'

# For loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [38]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
#uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [39]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [40]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Creating a Keyspace

In [41]:
# Create a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS cassproject
                   WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")

#### Setting Keyspace

In [42]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('cassproject')

#### Creating tables to run the following queries.
##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Creating a `song_albumn` table to answer the below question
##### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [43]:
Query1 = """CREATE TABLE IF NOT EXISTS song_albumn(sessionId int, 
            itemInSession int, artist text, songTitle text, songLength float, PRIMARY KEY(sessionId, itemInSession))"""

session.execute(Query1)

In [44]:
# Fetching CSV file that holds events data.
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
## INSERT data into song_albumn
        query = "INSERT INTO song_albumn(sessionId, itemInSession, artist, songTitle, songLength) "
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Selecting the table to verify that the data have been inserted

In [45]:
## Fetching results for  artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
quer1_result = "select artist, songTitle, songLength from song_albumn where sessionId = 338 and itemInSession = 4"
rows = session.execute(quer1_result)

for i in rows:
    print(i)

Row(artist='Faithless', songtitle='Music Matters (Mark Knight Dub)', songlength=495.30731201171875)


#### Creating `artist_albumn` table to answer the below question
##### Give me only the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [46]:

## Here we have used composite primary key (userId, sessionid) to uniquely identify each row in the table

# Creating Table artist_albumn
Query2 = """CREATE TABLE IF NOT EXISTS artist_albumn(userId int, sessionId int, itemInSession int, artist text, 
            songTitle text, firstName text, lastName text, PRIMARY KEY((userId, sessionid), itemInSession))"""
session.execute(Query2)


#Inserting records into table artist_albumn
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_albumn(userId, sessionId, itemInSession, artist, songTitle, firstName, lastName) "
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])) 
        
        
#Result of query2
quer2_result = "select artist, songTitle, firstName, lastName, itemInSession from artist_albumn where userid = 10 and sessionid = 182"
rows = session.execute(quer2_result)

for i in rows:
    print(i)

Row(artist='Down To The Bone', songtitle="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz', iteminsession=0)
Row(artist='Three Drives', songtitle='Greece 2000', firstname='Sylvie', lastname='Cruz', iteminsession=1)
Row(artist='Sebastien Tellier', songtitle='Kilometer', firstname='Sylvie', lastname='Cruz', iteminsession=2)
Row(artist='Lonnie Gordon', songtitle='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', iteminsession=3)


#### Creating table `music_albumn` to answer below question
##### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [47]:
## Creating Table music_albumn
Query3 = """CREATE TABLE IF NOT EXISTS music_albumn(song text, userId int, firstName text, lastName text, PRIMARY KEY(song, userId))"""
session.execute(Query3)


## Inserting records into table music_albumn
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_albumn(song, userId, firstName, lastName) "
        query = query + " VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4])) 
        
        
## Result of query3
quer3_result = "select song, firstName, lastName from music_albumn where song = 'All Hands Against His Own'"
rows = session.execute(quer3_result)

for i in rows:
    print(i)                    

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


#### Dropping the tables before closing out the sessions

In [48]:
## Drop the table before closing out the sessions so when the next time you try creating the table it doesnt throw error
session.execute("DROP TABLE song_albumn")
session.execute("DROP TABLE artist_albumn")
session.execute("DROP TABLE music_albumn")

#### Closing the session and cluster connection¶

In [49]:
session.shutdown()
cluster.shutdown()